In [55]:
import polars as pl

pl.Config.set_tbl_cols(200)
pl.Config.set_fmt_str_lengths(20)
pl.Config.set_tbl_width_chars(1000)
pl.Config.set_tbl_rows(200)

polars.config.Config

In [ ]:
df = pl.read_parquet('games.parquet')
print(df.columns)

team1 = 'PIT'
team2 = 'BOS'

df = df.filter(((pl.col('homeTeam') == team1) | (pl.col('awayTeam') == team1)) & ((pl.col('homeTeam') == team2) | (pl.col('awayTeam') == team2)))
df = df.with_columns(
  pl.when(pl.col('homeScore') > pl.col('awayScore')).then(pl.col('homeTeam')).otherwise(pl.col('awayTeam')).alias('winner')
)
df.group_by('winner').len()

In [ ]:
df = pl.read_parquet('games.parquet')

df = df.with_columns((pl.col('homePim') + pl.col('awayPim')).abs().alias('pim'))
df.sort(pl.col('pim'), descending=True)

In [60]:
import polars as pl

# Getting team stats
# team_stats = pl.read_parquet('team_stats.parquet')
# standings = pl.read_parquet('team_standings.parquet').unnest('teamName').rename({'default': 'teamFullName'})
# full_team_stats = team_stats.join(standings, on=('seasonId', 'teamFullName'))
# full_team_stats = full_team_stats.unnest('teamAbbrev').rename({'default': 'teamAbbrev'})

games = pl.read_parquet('games.parquet')
games = games.with_columns(
  pl.col('season').cast(pl.UInt64).alias('seasonId'),
  pl.when(pl.col('homeScore') > pl.col('awayScore')).then(pl.col('homeTeam')).otherwise(pl.col('awayTeam')).alias('winner')
)

playoffs = games.filter(pl.col('gameType') == 3)
playoffs = playoffs.with_columns(
  pl.col('gameId').cast(pl.String).str.head(-1).alias('series')
)
playoffs = playoffs.with_columns(
  (pl.col('winner') == pl.col('homeTeam').first()).cum_sum().over(pl.col('series')).shift(1).fill_null(0).alias('seriesHomeTeamWins'),
  (pl.col('winner') == pl.col('awayTeam').first()).cum_sum().over(pl.col('series')).shift(1).fill_null(0).alias('seriesAwayTeamWins'),
)

playoffs = playoffs.with_columns(
  pl.when(pl.col('homeTeam') == pl.col('homeTeam').first()).then(pl.col('seriesHomeTeamWins')).otherwise(pl.col('seriesAwayTeamWins')).alias('seriesHomeTeamWins'),
  pl.when(pl.col('awayTeam') == pl.col('awayTeam').first()).then(pl.col('seriesAwayTeamWins')).otherwise(pl.col('seriesHomeTeamWins')).alias('seriesAwayTeamWins')
)

playoffs = playoffs.with_columns(
  (pl.col('winner') == pl.col('homeTeam')).cast(pl.UInt64).alias('homeTeamWon'),
  (pl.col('winner') == pl.col('awayTeam')).cast(pl.UInt64).alias('awayTeamWon'),
)

playoffs = playoffs.select('gameId', 'gameType', 'seasonId', 'homeTeam', 'seriesHomeTeamWins', 'awayTeam', 'seriesAwayTeamWins', 'homeTeamWon', 'awayTeamWon')
playoffs = playoffs.with_columns(
  pl.when(pl.col('homeTeam') < pl.col('awayTeam')).then(pl.concat_str(pl.col('homeTeam'), pl.col('awayTeam'), separator='-')).otherwise(pl.concat_str(pl.col('awayTeam'), pl.col('homeTeam'), separator='-')).alias('matchup')
)


def aggregate_stats(df: pl.DataFrame, home: bool = True, matchup: bool = True) -> pl.DataFrame:
  df = df.filter(pl.col('gameType') == 2)
  for_prefix = 'home' if home else 'away'
  against_prefix = 'away' if home else 'home'
  if matchup:
    df = df.with_columns(
      pl.when(pl.col('homeTeam') < pl.col('awayTeam')).then(pl.concat_str(pl.col('homeTeam'), pl.col('awayTeam'), separator='-')).otherwise(pl.concat_str(pl.col('awayTeam'), pl.col('homeTeam'), separator='-')).alias('matchup')
    )
    groupby_cols = ('seasonId', 'matchup', f'{for_prefix}Team')
  else:
    groupby_cols = ('seasonId', f'{for_prefix}Team')
  return df.group_by(*groupby_cols).agg(
    pl.col(f'{for_prefix}Score').sum().alias('goals'),
    pl.col(f'{against_prefix}Score').sum().alias('goalsAgainst'),
    pl.col(f'{for_prefix}Sog').sum().alias('sog'),
    pl.col(f'{for_prefix}FaceoffWinningPctg').mean().alias('faceoffWinningPctg'),
    pl.col(f'{for_prefix}PowerPlay').str.split(by='/').list.first().cast(pl.UInt64).sum().alias('powerPlayGoals'),
    pl.col(f'{for_prefix}PowerPlay').str.split(by='/').list.last().cast(pl.UInt64).sum().alias('powerPlays'),
    pl.col(f'{for_prefix}Pim').sum().alias('pim'),
    pl.col(f'{for_prefix}Hits').sum().alias('hits'),
    pl.col(f'{for_prefix}BlockedShots').sum().alias('blockShots'),
    pl.col(f'{for_prefix}Giveaways').sum().alias('giveaways'),
    pl.col(f'{for_prefix}Takeaways').sum().alias('takeaways'),
    (pl.col('winner') == pl.col(f'{for_prefix}Team')).cast(pl.UInt8).sum().alias('wins'),
    (pl.col('winner') != pl.col(f'{for_prefix}Team')).cast(pl.UInt8).sum().alias('losses')
  ).rename({f'{for_prefix}Team': 'team'})

season_home_stats = aggregate_stats(games, matchup=False)
season_away_stats = aggregate_stats(games, home=False, matchup=False)
season_stats = pl.concat((season_home_stats, season_away_stats)).group_by('seasonId', 'team').agg(pl.all().sum())
season_stats = season_stats.with_columns(pl.col('faceoffWinningPctg') / 2)

home_stats = aggregate_stats(games, matchup=True)
away_stats = aggregate_stats(games, home=False, matchup=True)
matchup_stats = pl.concat((home_stats, away_stats)).group_by('seasonId', 'matchup', 'team').agg(pl.all().sum())
matchup_stats = matchup_stats.with_columns(pl.col('faceoffWinningPctg') / 2)

total_stats = matchup_stats.join(season_stats, on=('seasonId', 'team'), suffix='Season')

data = playoffs.join(total_stats, left_on=('seasonId', 'matchup', 'homeTeam'), right_on=('seasonId', 'matchup', 'team'), suffix='Home')
data = data.join(total_stats, left_on=('seasonId', 'matchup', 'awayTeam'), right_on=('seasonId', 'matchup', 'team'), suffix='Away')
# Formatting the data

data = data.sort(by='gameId')
X = data.select(pl.exclude(('gameId', 'seasonId', 'gameType', 'matchup', 'homeTeamWon', 'awayTeamWon')))
Y = data.select('homeTeamWon')
print(Y)

shape: (1_247, 1)
┌─────────────┐
│ homeTeamWon │
│ ---         │
│ u64         │
╞═════════════╡
│ 1           │
│ 1           │
│ 1           │
│ 0           │
│ 1           │
│ 0           │
│ 1           │
│ 0           │
│ 1           │
│ 0           │
│ 0           │
│ 1           │
│ 0           │
│ 0           │
│ 0           │
│ 0           │
│ 1           │
│ 1           │
│ 1           │
│ 1           │
│ 0           │
│ 0           │
│ 0           │
│ 0           │
│ 1           │
│ 0           │
│ 1           │
│ 1           │
│ 0           │
│ 1           │
│ 0           │
│ 1           │
│ 1           │
│ 1           │
│ 0           │
│ 0           │
│ 0           │
│ 0           │
│ 0           │
│ 1           │
│ 1           │
│ 0           │
│ 0           │
│ 0           │
│ 1           │
│ 1           │
│ 0           │
│ 0           │
│ 1           │
│ 0           │
│ 0           │
│ 1           │
│ 1           │
│ 0           │
│ 0           │
│ 1           │
│ 1   

In [ ]:
import xgboost as xgb